In [1]:
import cv2
import time

In [34]:
# cap = cv2.VideoCapture('1_20170406_132210.mp4')
cap = cv2.VideoCapture(1)
print cap.set(3, 1280.)
print cap.set(4, 720.)
print cap.get(3)
print cap.get(4)
time.sleep(1)
# cap.set(cv2.cv.CV_CAP_DSHOW, 700)
cap.set(cv2.cv.CV_CAP_PROP_FPS, 60)
count = 0
for _ in range(2000):
    ret, frame = cap.read()
    if ret:
#         print frame.shape
        cv2.imshow('dd', frame)
        key = cv2.waitKey(2)
        if key == ord('c'):
            break
    else:
        count += 1
        print 'fail'
    if count >= 10:
        break
cv2.destroyAllWindows()
cap.release()

True
True
640.0
480.0


In [17]:
cap.release()
cv2.destroyAllWindows()

In [27]:
dir(cap)

['__class__',
 '__delattr__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'get',
 'grab',
 'isOpened',
 'open',
 'read',
 'release',
 'retrieve',
 'set']

In [22]:
#!/usr/bin/env python
# -*- encoding: utf-8 -*-

from comtypes import client, GUID
import interfaces as dsifs
from uuids import ids, rids

qedit = client.GetModule("qedit.dll")
quartz = client.GetModule("quartz.dll")
MFVideoARMode_PreservePicture = 0x1

# Helper functions.
def get_name(byte_array):
    name = "".join(map(chr, byte_array))
    try:
        name = name[:name.index('\x00')]
    except ValueError:
        pass
    return name

def get_pin(afilter, pin_name):
    pin_enum = afilter.EnumPins()
    while True:
        cur_pin, fetched = pin_enum.Next(1)
        if fetched:
            pin_info = cur_pin.QueryPinInfo()
            cur_pin_name = get_name(pin_info.achName)
            if pin_name in cur_pin_name:
                return cur_pin
        else:
            cur_pin.Release()
            break
    print("Pin '%s' not found!" % pin_name)
    return None

def build_graph():
    graph = client.CreateObject(ids.CLSID_FilterGraph,
    interface=qedit.IFilterGraph)
    # Provided with a graph, create a builder
    builder = graph.QueryInterface(qedit.IGraphBuilder)
    capture = client.CreateObject(ids.CLSID_CaptureGraphBuilder2,
        interface=dsifs.ICaptureGraphBuilder2)

    # Filter chain xbar -> tuner -> a/v decoders -> a/v renderers.

    ##
    # Crossbar
    # Fetch the first one and set it up to Composite input.
    dev_enum_xbar = client.CreateObject(ids.CLSID_SystemDeviceEnum,
        interface=dsifs.ICreateDevEnum)
    xbar_enum = dev_enum_xbar.CreateClassEnumerator(
        GUID(ids.CLSID_VideoCrossbarCategory),
        dwFlags = 0)
    xbar_enum.Reset()
    xbar_moniker, success = xbar_enum.RemoteNext(1)
    if not success:
        raise IndexError("Couldn't find crossbar device.")
    xbar_bind = xbar_moniker.RemoteBindToObject(None, None,
        dsifs.IBaseFilter._iid_)
    xbar_filter = xbar_bind.QueryInterface(dsifs.IBaseFilter)

    # Setup
    xbar = xbar_filter.QueryInterface(dsifs.IAMCrossbar)
    outputs_count, inputs_count = xbar.get_PinCounts()
    for i in range(0, inputs_count):
        related_pin_idx, pin_type = xbar.get_CrossbarPinInfo(True, i)
        if pin_type == dsifs.PhysConn_Video_Composite:
            input_pin_index = i
            related_input_pin_idx = related_pin_idx
            break
    for i in range(0, outputs_count):
        related_pin_idx, pin_type = xbar.get_CrossbarPinInfo(False, i)
        if pin_type == dsifs.PhysConn_Video_VideoDecoder:
            output_pin_index = i
            related_output_pin_idx = related_pin_idx
            break
    # Route video pins
    if xbar.CanRoute(output_pin_index, input_pin_index) == 0:
        if xbar.Route(output_pin_index, input_pin_index) == 0:
            routed_to = xbar.get_IsRoutedTo(output_pin_index)
            if routed_to != input_pin_index:
                raise Exception("Couldn't route crossbar pins!.")
    # Route related (audio) pins
    if xbar.CanRoute(related_output_pin_idx, related_input_pin_idx) == 0:
        if xbar.Route(related_output_pin_idx, related_input_pin_idx) == 0:
            routed_to = xbar.get_IsRoutedTo(related_output_pin_idx)
            if routed_to != related_input_pin_idx:
                raise Exception("Couldn't route related crossbar pins!.")
    graph.AddFilter(xbar_filter, "Crossbar")

    ##
    # Capture (TV Tuner) device
    # Fetch the first one and set it up to NTSC.

    dev_enum_cap = client.CreateObject(ids.CLSID_SystemDeviceEnum,
        interface=dsifs.ICreateDevEnum)
    cap_enum = dev_enum_cap.CreateClassEnumerator(
        GUID(ids.CLSID_VideoInputDeviceCategory),
        dwFlags = 0)
    cap_enum.Reset()
    cap_moniker, success = cap_enum.RemoteNext(1)
    if not success:
        raise IndexError("Couldn't find capture device!.")
    cap_bind = cap_moniker.RemoteBindToObject(None, None,
        dsifs.IBaseFilter._iid_)
    cap_filter = cap_bind.QueryInterface(interface=dsifs.IBaseFilter)
    cap = cap_filter.QueryInterface(interface=dsifs.IAMAnalogVideoDecoder)
    cap.put_TVFormat(dsifs.AnalogVideo_NTSC_M)
    graph.AddFilter(cap_filter, "Video Capture")

    ##
    # Connect crossbar to capture.
    graph.ConnectDirect(get_pin(xbar_filter, "0: Video Decoder Out"),
        get_pin(cap_filter, "Analog Video In"), None)
    graph.ConnectDirect(get_pin(xbar_filter, "1: Audio Decoder Out"),
        get_pin(cap_filter, "Analog Audio In"), None)

    ##
    # ffdshow filters
    # video
    ffds_vidraw = client.CreateObject(ids.CLSID_FFDshowRawVideoFilter,
        interface=dsifs.IBaseFilter)
    graph.AddFilter(ffds_vidraw, "ffdshow raw video filter")
    # audio
    ffds_auddec = client.CreateObject(ids.CLSID_FFDshowAudioDecoder,
        interface=dsifs.IBaseFilter)
    graph.AddFilter(ffds_auddec, "ffdshow Audio Decoder")

    # link to capture filters.
    graph.ConnectDirect(get_pin(cap_filter, "Capture"),
        get_pin(ffds_vidraw, "In"), None)
    graph.ConnectDirect(get_pin(cap_filter, "Audio Out"),
        get_pin(ffds_auddec, "In"), None)

    ##
    # Renderers.
    # audio
    dsound_renderer = client.CreateObject(ids.CLSID_DSoundRender,
        interface=dsifs.IBaseFilter)
    graph.AddFilter(dsound_renderer, "Default DirectSound Device")
    # video
    enhanced_video = client.CreateObject(ids.CLSID_EnhancedVideoRenderer,
        interface=dsifs.IBaseFilter)
    graph.AddFilter(enhanced_video, "Enhanced Video Renderer")

    # link to ffdshow filters.
    graph.ConnectDirect(get_pin(ffds_vidraw, "Out"),
        get_pin(enhanced_video, "EVR Input0"), None)
    graph.ConnectDirect(get_pin(ffds_auddec, "Out"),
        get_pin(dsound_renderer, "Audio Input pin (rendered)"), None)

    # All done, return the EVR Video Display Control and Graph Media Control
    evr_service = enhanced_video.QueryInterface(dsifs.IMFGetService)
    vdc_service = evr_service.GetService(GUID(ids.MR_VIDEO_RENDER_SERVICE),
        dsifs.IMFVideoDisplayControl._iid_)
    evr_videocontrol = vdc_service.QueryInterface(dsifs.IMFVideoDisplayControl)
    media_control = graph.QueryInterface(quartz.IMediaControl)
    return (media_control, evr_videocontrol)

if __name__ == "__main__":
    gr = client.CreateObject(ids.CLSID_FilterGraph,
        interface=qedit.IFilterGraph)
    print(build_graph(gr))

ImportError: No module named uuids

In [2]:
import chptocr

In [3]:
chptocr.coordinates(im[80:100, 30:77])

[]

# PyOCR

In [3]:
from PIL import Image
import sys

import pyocr
import pyocr.builders

tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("No OCR tool found")
    sys.exit(1)
# The tools are returned in the recommended order of usage
tool = tools[0]
print("Will use tool '%s'" % (tool.get_name()))
# Ex: Will use tool 'libtesseract'

langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))
lang = langs[0]
print("Will use lang '%s'" % (lang))
# Ex: Will use lang 'fra'
# Note that languages are NOT sorted in any way. Please refer
# to the system locale settings for the default language
# to use.

Will use tool 'Tesseract (sh)'
Available languages: eng, osd, wid
Will use lang 'eng'


In [10]:
img = Image.open('2.BMP')

In [ ]:
import cv2
import numpy as np
img = cv2.imread('1.BMP')

img_r = np.array([
    [
        255 if pixel[2]*(pixel[2]>3*pixel[0])*(pixel[2]>3*pixel[1]) > 80 else 0
        for pixel in row
    ]
    for row in img
], dtype='uint8')

cv2.imshow('dd', img_r)
cv2.waitKey(0)
cv2.destroyAllWindows()

# print img

In [4]:

image = Image.fromarray(img_r)
txt = tool.image_to_string(
    image,
    lang='eng',#'wid',
    builder=pyocr.builders.TextBuilder()
)
# txt is a Python string
print txt

A - 31.762 um


In [19]:

txt = tool.image_to_string(
    Image.open('2.BMP'),
    lang='eng',#'wid',
    builder=pyocr.builders.TextBuilder()
)
# txt is a Python string
print txt

In [21]:
word_boxes = tool.image_to_string(
    Image.open('S4_SNAP.BMP'),
    lang="eng",
    builder=pyocr.builders.WordBoxBuilder()
)
# list of box objects. For each box object:
#   box.content is the word in the box
#   box.position is its position on the page (in pixels)
#
# Beware that some OCR tools (Tesseract for instance)
# may return empty boxes
for word_box in word_boxes:
    print word_box.content, word_box.position

cm: ((274, 7), (302, 17))
550 ((308, 7), (333, 17))
Eﬁﬁﬁ ((339, 5), (393, 19))
Vnfu ((660, 6), (681, 17))
um ((749, 8), (772, 17))
4 ((833, 8), (839, 17))
7 ((856, 8), (861, 14))
  ((0, 2), (8, 23))
DN:lCAMlRD/2017020153-A1D01/2017020153-1-1-1/CHP ((8, 15), (512, 45))
2017/2/la ((725, 33), (787, 43))
7:: ((804, 32), (828, 44))
02 ((833, 33), (847, 42))
49 ((853, 33), (867, 42))
44 ((873, 33), (887, 42))
Ségtﬁs ((933, 33), (981, 62))
A ((81, 62), (93, 65))
7 ((958, 50), (962, 61))
E ((69, 63), (77, 75))
WW ((82, 62), (121, 76))
iﬁﬁreﬁ ((252, 71), (306, 85))
ME ((441, 70), (469, 84))
mm: ((603, 70), (657, 84))
mfﬁﬁ ((778, 69), (832, 83))
,, ((984, 81), (996, 88))
T ((14, 86), (21, 95))
1 ((65, 86), (71, 96))
M ((94, 86), (102, 95))
253 ((158, 86), (180, 96))
can ((434, 90), (458, 99))
ON ((466, 90), (482, 99))
Typ ((3, 103), (23, 115))
vn ((53, 102), (71, 112))
EL ((94, 103), (107, 112))
mason ((142, 102), (180, 112))
9 ((212, 106), (218, 114))
as ((224, 106), (236, 114))
3; ((241, 106),

In [35]:
line_and_word_boxes = tool.image_to_string(
    Image.open('SNAP2.png'), lang="eng",
    builder=pyocr.builders.LineBoxBuilder()
)
# list of line objects. For each line object:
#   line.word_boxes is a list of word boxes (the individual words in the line)
#   line.content is the whole text of the line
#   line.position is the position of the whole line on the page (in pixels)
#
# Beware that some OCR tools (Tesseract for instance)
# may return empty boxes

# Digits - Only Tesseract (not 'libtesseract' yet !)
for s in  line_and_word_boxes:
    print s.content

W]
3 150
046
530


In [37]:
digits = tool.image_to_string(
    Image.open('SNAP2.png'),
    lang=lang,
    builder=pyocr.tesseract.DigitBuilder()
)
# digits is a python string
print digits

1
3 150

046
530


In [1]:
print 12*3600

43200


In [2]:
import numpy as np

In [9]:
string = "abcdef"
for word in string:
    print format(ord(word), 'b')

1100001
1100010
1100011
1100100
1100101
1100110
